In [3]:
import os
import email
import imaplib
import configparser
import re

In [4]:
def find_encoding_info(txt):
    info = email.header.decode_header(txt)
    s, encoding = info[0]
    return s, encoding


In [5]:
config = configparser.ConfigParser()
config.read('D:\개인\Work\config.ini')

['D:\\개인\\Work\\config.ini']

In [6]:
imap_url = imaplib.IMAP4_SSL('imap.gmail.com')

In [7]:
imap_url.login(config['Gmail']['id'], config['Gmail']['pw'])

('OK', [b'kimgs243@gmail.com authenticated (Success)'])

In [8]:
imap_url.select('INBOX')

('OK', [b'3'])

In [10]:
imap_url.list()

('OK',
 [b'(\\HasNoChildren) "/" "INBOX"',
  b'(\\HasChildren \\Noselect) "/" "[Gmail]"',
  b'(\\Flagged \\HasNoChildren) "/" "[Gmail]/&vMTUXNO4ycDVaA-"',
  b'(\\HasNoChildren \\Sent) "/" "[Gmail]/&vPSwuNO4ycDVaA-"',
  b'(\\HasNoChildren \\Junk) "/" "[Gmail]/&wqTTONVo-"',
  b'(\\Drafts \\HasNoChildren) "/" "[Gmail]/&x4TC3Lz0rQDVaA-"',
  b'(\\All \\HasNoChildren) "/" "[Gmail]/&yATMtLz0rQDVaA-"',
  b'(\\HasNoChildren \\Important) "/" "[Gmail]/&yRHGlA-"',
  b'(\\HasNoChildren \\Trash) "/" "[Gmail]/&1zTJwNG1-"',
  b'(\\HasNoChildren) "/" "&x4TC3A- &vPStANVo-"'])

In [11]:
result, data = imap_url.search(None, 'ALL')

In [19]:
data

decoration: none; color: #757575; -webkit-text-=\r\nsize-adjust: none; -ms-text-size-adjust: none; font-weight: bold; margin: 0=\r\n; padding: 0;">Spotify Podcasts</a></p>\r\n                  </td>\r\n                </tr>\r\n              </table>\r\n\r\n            </td>\r\n          </tr>\r\n          <tr style=3D"border-collapse: collapse; margin: 0; padding: 0;">\r\n            <td class=3D"extra-links" style=3D"border-collapse: collapse; m=\r\nargin: 0; padding: 0;">\r\n              <p style=3D"font-family: helvetica, sans-serif; text-decorati=\r\non: none; color: #757575; -webkit-text-size-adjust: none; -ms-text-size-adj=\r\nust: none; font-weight: 300; display: inline; font-size: 15px; line-height:=\r\n 24px; margin: 0; padding: 0;">\r\n                as well as on <a href=3D"http://email.n.dribbble.com/c/eJxN=\r\njs1uhSAQhZ8GdxgcEGXBoklzX6PhZ_CSOmoA7_NXm7ZpMouT851Mvmijhgl0l21IcxpTmjmIELg=\r\nyANxIbfg8jkJPAg2aiSmx9bFk7_2Kfdipe1oTR9SoYgKfZqfFAGI0wjmvg1Npct1qn60dlck3Bo=\r\n_r9

In [22]:
all_email = data[0].split()
print(all_email)

AttributeError: 'tuple' object has no attribute 'split'

In [24]:
raw_email = data[0][1]

In [25]:
raw_email_string = raw_email.decode('UTF-8')

In [27]:
email_message = email.message_from_string(raw_email_string)

In [31]:
email_message['Subject']

'=?utf-8?q?=F0=9F=8E=A7_Design_Like_a_Kid?='

In [15]:
for mail in all_email:
    result, data = imap_url.fetch(mail, '(RFC822)')
    raw_email = data[0][1]
    raw_email_string = raw_email.decode('UTF-8')
    email_message = email.message_from_string(raw_email_string)

    # 메일 정보
    print('From: ', email_message['From'])
    print('Sender: ', email_message['Sender'])
    print('To: ', email_message['To'])
    print('Date: ', email_message['Date'])

    subject, encode = find_encoding_info(
        email_message['Subject'])
    print('Subject', subject)

    message = ''

    print('[Message]')

From:  GwiSoo Kim <kgs243@outlook.com>
Sender:  None
To:  GwiSoo Kim <kimgs243@gmail.com>
Date:  Tue, 20 Oct 2020 08:00:54 +0000
Subject FW: Atachfile's
[Message]
From:  "Medium Daily Digest" <noreply@medium.com>
Sender:  None
To:  kimgs243@gmail.com
Date:  Wed, 21 Oct 2020 08:40:00 +0000 (UTC)
Subject Burn Fat in Your Sleep, Start Tonight | Tim Rees
[Message]
From:  Dribbble <no-reply@n.dribbble.com>
Sender:  no-reply@n.dribbble.com
To:  '' <kimgs243@gmail.com>
Date:  Wed, 21 Oct 2020 13:42:59 +0000
Subject b'\xf0\x9f\x8e\xa7 Design Like a Kid'
[Message]


In [1]:
if email_message.is_multipart():
    for part in email_message.get_payload():
        if part.get_content_type() == 'text/plain':
            bytes = part.get_payload(decode=True)
            encode = part.get_content_charset()
            message = message + str(bytes, encode)
else:
    if email_message.get_content_type() == 'text/plain':
        bytes = email_message.get_payload(decode=True)
        encode = email_message.get_content_charset()
        message = str(bytes, encode)
print(message)

NameError: name 'email_message' is not defined

In [60]:
for part in email_message.walk():
    if part.get_content_maintype() == 'multipart':
        continue
    if part.get('Content-Disposition') is None:
        continue
    file_name = part.get_filename()

    # file_name = "".join(
    #     i for i in file_name if i not in "\/:*?<>|")

    # file_name = file_name.replace('UTF-8B', '')
    # file_name = file_name.replace('utf-8B', '')
    # print(file_name)

    if bool(file_name):
        file_path = os.path.join(
            'D:/개인/Work/Download', file_name)
        if not os.path.isfile(file_path):
            fp = open(file_path, 'wb')
            fp.write(part.get_payload(decode=True))
            fp.close()
else:
    continue

SyntaxError: 'continue' not properly in loop (<ipython-input-60-dabec65ac40d>, line 26)